In [1]:
import torch
import torch.nn as nn

Calculate the sizes for torch conv2dTranspose layers to produce a desired output size from
a provided input size.


In [2]:
def calc_size( dim, layer ) :
    return ( dim-1 ) * layer['stride'] - ( 2 * layer['padding'] ) + layer['kernel'] + layer['out_padding'] 

In [3]:
def find_size( insz, outsz, depth=0 ) :
    if depth < 4 :
        layer = {}
        for layer['stride'] in range( 1,3 ) :
            for layer['kernel'] in range( 2,11,2 ) :
                for layer['padding'] in range( 0,(layer['kernel']+1)>>1 ) :
                    for layer['out_padding'] in range( 0,layer['stride'] ) :
                        sz = calc_size( insz, layer )
                        if sz==outsz :
                            yield [ (layer, sz) ]
                        elif sz>insz :
                            for r in find_size( sz, outsz, depth+1 ) :
                                r.append( (layer, sz) )
                                yield r

In [6]:
input = 6
output = 96

ge = find_size( input, output )
n = 0
for r in ge :
    r.reverse()
    n = n + 1
    if n > 5 :
        break
    print( "------------", n , "------------" )
    print( "layers=[")
    for l in r :        
        print( "nn.ConvTranspose2d( nc, nc, " + str( l[0]['kernel'] )
              + ", stride=" + str( l[0]['stride'] )
              + ", padding=" + str( l[0]['padding'] )
              + ", output_padding=" + str( l[0]['out_padding'] ) 
              + ", bias=False), # out size " + str( l[1] ) )
    print( "]" )
        
if n==0 :
    print( "No solution" )


------------ 1 ------------
layers=[
nn.ConvTranspose2d( nc, nc, 2, stride=1, padding=0, output_padding=0, bias=False), # out size 7
nn.ConvTranspose2d( nc, nc, 6, stride=2, padding=0, output_padding=0, bias=False), # out size 18
nn.ConvTranspose2d( nc, nc, 10, stride=2, padding=0, output_padding=0, bias=False), # out size 44
nn.ConvTranspose2d( nc, nc, 10, stride=2, padding=0, output_padding=0, bias=False), # out size 96
]
------------ 2 ------------
layers=[
nn.ConvTranspose2d( nc, nc, 2, stride=1, padding=0, output_padding=0, bias=False), # out size 7
nn.ConvTranspose2d( nc, nc, 6, stride=2, padding=0, output_padding=0, bias=False), # out size 18
nn.ConvTranspose2d( nc, nc, 10, stride=2, padding=0, output_padding=1, bias=False), # out size 45
nn.ConvTranspose2d( nc, nc, 8, stride=2, padding=0, output_padding=0, bias=False), # out size 96
]
------------ 3 ------------
layers=[
nn.ConvTranspose2d( nc, nc, 2, stride=1, padding=0, output_padding=0, bias=False), # out size 7
nn.ConvTrans

This is a sample test for the results.
This skeleton can be used to verify the outputs. Overwrite the ```nn.Sequential( ... )```  
content with one copied from above.


In [32]:
class Squash3Dto1D( nn.Module ) :
    def forward( self, X ) :
        return X.view( X.size(0), X.size(1) * X.size(2) * X.size(3) )


In [34]:
nc = 1
h = 6
w = 6
batch = 17
X = torch.randn( batch, nc, h, w )
    
model = nn.Sequential(
    nn.ConvTranspose2d( nc, nc, 2, stride=1, padding=0, output_padding=0, bias=False), # out size 7
    nn.ConvTranspose2d( nc, nc, 6, stride=2, padding=0, output_padding=1, bias=False), # out size 19
    nn.ConvTranspose2d( nc, nc, 8, stride=2, padding=0, output_padding=1, bias=False), # out size 45
    nn.ConvTranspose2d( nc, nc, 8, stride=2, padding=0, output_padding=0, bias=False), # out size 96
    Squash3Dto1D(),
    nn.Linear( 96*96, 4 )
)

y=model.forward(X)
print ( X.size(), y.size() )

torch.Size([17, 1, 6, 6]) torch.Size([17, 4])
